In [1]:
import galprime as gp

from astropy.io import fits


In [4]:
sim = gp.GPrime()


In [6]:
cutouts = gp.Cutouts.from_file("../pdata/cutouts_i.fits", verbose=True)

AttributeError: module 'galprime' has no attribute 'Cutouts'

In [ ]:
print()